<a href="https://colab.research.google.com/github/chaenii989/Project_2_Impacts_on_US_Housing/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning and Transforming Housing Data

## Import and Read

In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
import datetime as dt
from functools import reduce

In [2]:
# Read csv

# 30-year fixed rate mortgage quarterly average (FRED)
# 1990 to 2021, float, rounding required, remove 7/1/2021 (just showing ".")
interest_rate_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/MORTGAGE30US.csv')

# Housing authorized, construction not started
# 1999 to 2021, convert to integers
authorized_not_started_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/authorized_not_started_AUTHNOTTSA.csv')

# Housing authorized, construction started
# 1990 to 2021, convert to integers
authorized_started_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/authorized_started_HOUST.csv')

# Average home price
# 1963 to 2021, convert to integers
average_home_price_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/average_home_price.csv')

# Homeownership rate
# 1990 to 2021, float, nothing further needed with data type/rounding
homeownership_rate_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/homeownership_rate_RSAHORUSQ156S.csv')

# Lumber price index
# 1990 to 2021, float, rounding required
lumber_price_index_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/lumber_price_19900101_20210601.csv')

# House supply
# 1990 to 2021, float, nothing further needed wiht data type/rounding
house_supply_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/monthly_supply_houses_us_MSACSR.csv')

# New housing permits
# 1990 to 2021, convert to integers
new_housing_permits_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/new_housing_permits_PERMITNSA.csv')

# Steel price index
# 1990 to 2021, float, rounding required
steel_price_index_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/steel_price_index.csv')

# Under construction
# 1990 to 2021, convert to integers
under_construction_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/under_counstruction_UNDCONTSA.csv')

# Units completed
# 1990 to 2021, convert to integers
units_completed_df = pd.read_csv(r'https://raw.githubusercontent.com/chaenii989/Final_Project_What_is_Affecting_US_Housing_Market/main/resources/units_completed_COMPUTNSA.csv')

## Clean Each Dataset

In [3]:
# Clean interest rate DataFrame
# Check data types
interest_rate_df.dtypes
# Change column names
interest_rate_df = interest_rate_df.rename(columns={"MORTGAGE30US":"interest_rate","DATE":"date"})
# Drop entire row for 7/1/21 which reads "." 
interest_rate_df.drop(interest_rate_df.index[(interest_rate_df["date"] == "2021-07-01")],axis=0,inplace=True)
# Convert object to float
interest_rate_df["interest_rate"] = interest_rate_df["interest_rate"].astype(float, errors = 'raise')
# Round to two decimal places
interest_rate_df["interest_rate"] = interest_rate_df["interest_rate"].round(decimals=2)
# Check for any null values in entire DF
interest_rate_df.isnull().values.any()

interest_rate_df.head()

,date,interest_rate
0,1990-01-01,10.13
1,1990-04-01,10.32
2,1990-07-01,10.10
3,1990-10-01,9.96
4,1991-01-01,9.50


In [4]:
# Clean housing authorized, not started DataFrame
# Check data types
authorized_not_started_df.dtypes
# Change column names
authorized_not_started_df = authorized_not_started_df.rename(columns={"AUTHNOTTSA":"units_authorized_not_started","DATE":"date"})
# Check for any null values in entire DF
authorized_not_started_df.isnull().values.any()

authorized_not_started_df.head()

,date,units_authorized_not_started
0,1999-01-01,134.0
1,1999-04-01,149.0
2,1999-07-01,143.0
3,1999-10-01,140.0
4,2000-01-01,141.0


In [5]:
# Clean housing authorized, started DataFrame
# Check data types
authorized_started_df.dtypes
# Change column names
authorized_started_df = authorized_started_df.rename(columns={"HOUST":"units_authorized_started","DATE":"date"})
# Check for any null values in entire DF
authorized_started_df.isnull().values.any()

authorized_started_df.head()

,date,units_authorized_started
0,1990-01-01,1289.0
1,1990-04-01,1177.0
2,1990-07-01,1110.0
3,1990-10-01,969.0
4,1991-01-01,921.0


In [6]:
# Clean average home price DataFrame
# Check data types
average_home_price_df.dtypes
# Change column names
average_home_price_df = average_home_price_df.rename(columns={"MSPUS":"avg_home_price","DATE":"date"})
# Check for any null values in entire DF
average_home_price_df.isnull().values.any()

average_home_price_df.head()

,date,avg_home_price
0,1963-01-01,17800.0
1,1963-04-01,18000.0
2,1963-07-01,17900.0
3,1963-10-01,18500.0
4,1964-01-01,18500.0


In [7]:
# Clean homeownership rate DataFrame
# Check data types
homeownership_rate_df.dtypes
# Change column names
homeownership_rate_df = homeownership_rate_df.rename(columns={"RSAHORUSQ156S":"homeownership_rate","DATE":"date"})
# Check for any null values in entire DF
homeownership_rate_df.isnull().values.any()

homeownership_rate_df.head()

,date,homeownership_rate
0,1990-01-01,64.1
1,1990-04-01,63.9
2,1990-07-01,63.9
3,1990-10-01,64.0
4,1991-01-01,64.0


In [8]:
# Clean average home price DataFrame
# Check data types
average_home_price_df.dtypes
# Change column names
average_home_price_df = average_home_price_df.rename(columns={"MSPUS":"avg_home_price","DATE":"date"})
# Convert object to integer
average_home_price_df["avg_home_price"] = average_home_price_df["avg_home_price"].astype(int, errors = 'raise')
# Check for any null values in entire DF
average_home_price_df.isnull().values.any()

average_home_price_df.head()

,date,avg_home_price
0,1963-01-01,17800
1,1963-04-01,18000
2,1963-07-01,17900
3,1963-10-01,18500
4,1964-01-01,18500


In [9]:
# Clean lumber price index DataFrame
# Check data types
lumber_price_index_df.dtypes
# Change column names
lumber_price_index_df = lumber_price_index_df.rename(columns={"lumber_price":"lumber_price_index"})
# Round to two decimal places
lumber_price_index_df["lumber_price_index"] = lumber_price_index_df["lumber_price_index"].round(decimals=2)
# Check for any null values in entire DF
lumber_price_index_df.isnull().values.any()

lumber_price_index_df.head()

,date,lumber_price_index
0,1990-01-01,55.58
1,1990-04-01,56.64
2,1990-07-01,54.72
3,1990-10-01,51.38
4,1991-01-01,51.48


In [10]:
# Clean house supply DataFrame
# Check data types
house_supply_df.dtypes
# Change column names
house_supply_df = house_supply_df.rename(columns={"MSACSR":"house_supply","DATE":"date"})
# Check for any null values in entire DF
house_supply_df.isnull().values.any()

house_supply_df.head()

,date,house_supply
0,1990-01-01,7.8
1,1990-04-01,7.9
2,1990-07-01,8.4
3,1990-10-01,8.5
4,1991-01-01,7.3


In [11]:
# Clean new housing permits DataFrame
# Check data types
new_housing_permits_df.dtypes
# Change column names
new_housing_permits_df = new_housing_permits_df.rename(columns={"PERMIT":"new_housing_permits","DATE":"date"})
# Check for any null values in entire DF
new_housing_permits_df.isnull().values.any()

new_housing_permits_df.head()

,date,new_housing_permits
0,1990-01-01,1246.0
1,1990-04-01,1108.0
2,1990-07-01,976.0
3,1990-10-01,861.0
4,1991-01-01,911.0


In [12]:
# Clean steel price index DataFrame
# Check data types
steel_price_index_df.dtypes
# Change column names
steel_price_index_df = steel_price_index_df.rename(columns={"WPU101":"steel_price_index","DATE":"date"})
# Round to two decimal places
steel_price_index_df["steel_price_index"] = steel_price_index_df["steel_price_index"].round(decimals=2)
# Check for any null values in entire DF
steel_price_index_df.isnull().values.any()

steel_price_index_df.head() 

,date,steel_price_index
0,1990-01-01,116.47
1,1990-04-01,117.47
2,1990-07-01,117.87
3,1990-10-01,117.07
4,1991-01-01,116.47


In [13]:
# Clean under construction DataFrame
# Check data types
under_construction_df.dtypes
# Change column names
under_construction_df = under_construction_df.rename(columns={"UNDCONTSA":"under_construction","DATE":"date"})
# Check for any null values in entire DF
under_construction_df.isnull().values.any()

under_construction_df.head()

,date,under_construction
0,1990-01-01,885.0
1,1990-04-01,846.0
2,1990-07-01,793.0
3,1990-10-01,745.0
4,1991-01-01,678.0


In [14]:
# Clean units completed DataFrame
# Check data types
units_completed_df.dtypes
# Change column names
units_completed_df = units_completed_df.rename(columns={"COMPUTSA":"units_completed","DATE":"date"})
# Check for any null values in entire DF
units_completed_df.isnull().values.any()

units_completed_df.head()

,date,units_completed
0,1990-01-01,1345.0
1,1990-04-01,1263.0
2,1990-07-01,1312.0
3,1990-10-01,1173.0
4,1991-01-01,1176.0


## Merge All .csv Files

In [15]:
dfs_to_merge = [interest_rate_df, 
               authorized_not_started_df, 
               authorized_started_df, 
               average_home_price_df, 
               homeownership_rate_df, 
               lumber_price_index_df, 
               house_supply_df,
               new_housing_permits_df,
               steel_price_index_df, 
               under_construction_df, 
               units_completed_df
              ]

In [16]:
housing_combined_df = reduce(lambda  left,right: pd.merge(left,right,on=["date"],
                                            how="outer"), dfs_to_merge)
housing_combined_df.head()

,date,interest_rate,units_authorized_not_started,units_authorized_started,avg_home_price,homeownership_rate,lumber_price_index,house_supply,new_housing_permits,steel_price_index,under_construction,units_completed
0,1990-01-01,10.13,NaN,1289.0,123900.0,64.1,55.58,7.8,1246.0,116.47,885.0,1345.0
1,1990-04-01,10.32,NaN,1177.0,126800.0,63.9,56.64,7.9,1108.0,117.47,846.0,1263.0
2,1990-07-01,10.10,NaN,1110.0,117000.0,63.9,54.72,8.4,976.0,117.87,793.0,1312.0
3,1990-10-01,9.96,NaN,969.0,121500.0,64.0,51.38,8.5,861.0,117.07,745.0,1173.0
4,1991-01-01,9.50,NaN,921.0,120000.0,64.0,51.48,7.3,911.0,116.47,678.0,1176.0


In [17]:
# Check data types
housing_combined_df.dtypes
# Convert object to date
housing_combined_df["date"] = pd.to_datetime(housing_combined_df["date"]).dt.normalize()
# Convert objects to integer
housing_combined_df["units_authorized_not_started"] = housing_combined_df["units_authorized_not_started"].astype(pd.Int32Dtype())
housing_combined_df["units_authorized_started"] = housing_combined_df["units_authorized_started"].astype(pd.Int32Dtype())
housing_combined_df["avg_home_price"] = housing_combined_df["avg_home_price"].astype(pd.Int32Dtype())
housing_combined_df["new_housing_permits"] = housing_combined_df["new_housing_permits"].astype(pd.Int32Dtype())
housing_combined_df["under_construction"] = housing_combined_df["under_construction"].astype(pd.Int32Dtype())
housing_combined_df["units_completed"] = housing_combined_df["units_completed"].astype(pd.Int32Dtype())
housing_combined_df.dtypes
housing_combined_df.head()

,date,interest_rate,units_authorized_not_started,units_authorized_started,avg_home_price,homeownership_rate,lumber_price_index,house_supply,new_housing_permits,steel_price_index,under_construction,units_completed
0,1990-01-01,10.13,<NA>,1289,123900,64.1,55.58,7.8,1246,116.47,885,1345
1,1990-04-01,10.32,<NA>,1177,126800,63.9,56.64,7.9,1108,117.47,846,1263
2,1990-07-01,10.10,<NA>,1110,117000,63.9,54.72,8.4,976,117.87,793,1312
3,1990-10-01,9.96,<NA>,969,121500,64.0,51.38,8.5,861,117.07,745,1173
4,1991-01-01,9.50,<NA>,921,120000,64.0,51.48,7.3,911,116.47,678,1176


In [18]:
# Sort by date
housing_combined_df = housing_combined_df.sort_values(by="date", ascending=True)
housing_combined_df

,date,interest_rate,units_authorized_not_started,units_authorized_started,avg_home_price,homeownership_rate,lumber_price_index,house_supply,new_housing_permits,steel_price_index,under_construction,units_completed
126,1963-01-01,NaN,<NA>,<NA>,17800,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>
127,1963-04-01,NaN,<NA>,<NA>,18000,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>
128,1963-07-01,NaN,<NA>,<NA>,17900,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>
129,1963-10-01,NaN,<NA>,<NA>,18500,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>
130,1964-01-01,NaN,<NA>,<NA>,18500,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
121,2020-04-01,3.24,164,1273,322600,68.1,100.00,4.3,1296,203.90,1187,1243
122,2020-07-01,2.95,178,1448,337500,67.3,145.00,3.5,1589,202.17,1218,1426
123,2020-10-01,2.76,187,1661,358700,65.7,147.59,3.8,1758,216.40,1264,1386
124,2021-01-01,2.88,229,1725,347500,65.6,183.76,4.2,1755,266.97,1307,1497


In [19]:
# Create new csv
housing_combined_df.to_csv('etl/housing_combined_data.csv', index=False)